In [1]:
%matplotlib inline

import math
import pytz 
import traceback
import time
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import cPickle as pickle

In [2]:
%run src/data/helper.py

In [3]:
start_time = time.time()

with open('data/parsed/stations_dataset_final.p', 'rb') as f:
    stations = pickle.load(f)

with open('data/parsed/readings_clean.p', "rb") as f:
    readings = pickle.load(f)

end_time = time.time()
print 'Opening redistribution data took %s' % (end_time - start_time)

Opening redistribution data took 279.28757906


## Split Dataset

In [4]:
split_training = lambda df: df[datetime(2016,5,15,0,0,0,0):datetime(2016,6,12,23,59,59,999999)]
split_validation = lambda df: df[datetime(2016,6,13,0,0,0,0):datetime(2016,6,19,23,59,59,999999)]
split_test = lambda df: df[datetime(2016,5,20,0,0,0,0):datetime(2016,6,26,23,59,59,999999)]

In [5]:
def split_datasets(df, station_id):
    station_df = df.loc[station_id]
    training = split_training(station_df)
    validation = split_validation(station_df)
    test = split_test(station_df)
    
    return training, validation, test

## Model Definitions

In [6]:
import sys

def clip_and_round(arr):
    arr = np.clip(arr, 0, sys.maxint)
    return np.round(arr)

In [7]:
import inspect

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error

from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import IntVector, Formula

pandas2ri.activate()

r = robjects.r
base = importr('base')
stats = importr('stats')
mgcv = importr('mgcv')

class GAMRegressor(BaseEstimator, RegressorMixin):      
    last_data = None

    def __init__(self, features=None, formula_str=None,
                 FogTMinus2_parametric=None, RainTMinus2_parametric=None,
                 DistNbBikes_parametric=None, CollNbBikes_parametric=None, 
                 TempTMinus2AndHumidityTMinus2_sp=None,
                 NbBikesTMinus2_sp=None, NbBikesTMinus3_sp=None, 
                 NbBikesTMinus12_sp=None, NbBikesTMinus18_sp=None,             
                 TimeOfDay_1_sp=None, TimeOfDay_2_sp=None, TimeOfDay_3_sp=None,
                 TimeOfDay_1_by=None, TimeOfDay_2_by=None, TimeOfDay_3_by=None):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        self.args_to_set = []
        for arg, val in values.items():
            # save a list of the arguments
            if arg != 'features' and arg != 'formula_str':
                self.args_to_set.append(arg)
            setattr(self, arg, val)

    def fit(self, X, y=None): 
        if self.formula_str is None:
            features_dicts = self.build_features_dicts()
            self.formula_str = self.build_formula_str(features_dicts)       
            
        GAMRegressor.last_data=X
            
        frm = Formula(self.formula_str)
        self.gam = mgcv.gam(frm, data=X)
        
        return self

    def predict(self, X):
        assert (self.gam is not None), "GAM must be set"
        p_val = clip_and_round(stats.predict(self.gam, newdata=X))
        return p_val
    
    def score(self, X):
        p_val = self.predict(X)
        y_val = X.NbBikes
        rmse = mean_squared_error(y_val, p_val)**0.5
        return rmse * (-1)
    
    def build_features_dicts(self):
        assert (self.features is not None), "features must be set"
        
        # initialize the dictionaries
        features_dicts = {}
        for feature in self.features:
            features_dicts[feature] = {
                'name': feature,
                'bs': 'tp',
                'sp': None,
                'by': None,
                'k': None,
                'parametric': False
            }
            
        # set parameter values
        for arg in self.args_to_set:
            val = getattr(self, arg)
            if val is None:
                continue
            feature, parameter = arg.rsplit('_',1)
            features_dicts[feature][parameter] = val
            
        return features_dicts
    
    def build_formula_str(self, features_dicts):
        formula = 'NbBikes ~ '
        for feature, feature_dict in features_dicts.iteritems():
            if feature_dict['parametric']:
                formula += '%(name)s+' % feature_dict
                continue
                                
            tokens = feature_dict['name'].split('_')
            name, index = (tokens[0],None) if len(tokens) == 1 else (tokens[0], tokens[1])
            formula += "s(%s" % name.replace('And', ',')
            
            if feature_dict['bs'] is not None:
                formula += ", bs='%s'" % feature_dict['bs']
            if feature_dict['sp'] is not None:
                formula += ", sp=%f" % feature_dict['sp']
            if feature_dict['by'] is not None:
                formula += ", by=%s" % feature_dict['by']
            if feature_dict['k'] is not None:
                formula += ", k=%s" % feature_dict['k']
                
            formula += ")+" % feature_dict
        return formula[:-1]
    
class LRegressor(BaseEstimator, RegressorMixin):  
    def __init__(self, formula_str):
    	self.formula_str = formula_str

    def fit(self, X, y=None):            
        self.lr = stats.lm(Formula(self.formula_str), data=X)        
        return self

    def predict(self, X):
        assert (self.lr is not None), "LR must be set"
        p_val = clip_and_round(stats.predict(self.lr, newdata=X))
        return p_val
    
    def score(self, X):
        p_val = self.predict(X)
        y_val = X.NbBikes
        rmse = mean_squared_error(y_val, p_val)**0.5
        return rmse * (-1)   

In [8]:
from sklearn.linear_model import LinearRegression

def fit_and_predict_lm(training, validation, test, formula):
    lm = LRegressor(formula_str=formula)
    lm.fit(training)
    return lm, clip_and_round(lm.predict(training)), clip_and_round(lm.predict(validation)), clip_and_round(lm.predict(test))

In [9]:
def fit_and_predict_gam(training, validation, test, formula):
    gam = GAMRegressor(formula_str=formula)
    gam.fit(training)
    return gam, clip_and_round(gam.predict(training)), clip_and_round(gam.predict(validation)), clip_and_round(gam.predict(test))

In [10]:
def fit_and_predict_hist_avg(training, validation, test):
    return None, training.HistAvg, validation.HistAvg, test.HistAvg

In [11]:
def model(df, station_ids, gam_formula, lr_formula, pred_col):
    results = []

    for station_id in station_ids:
        print 'Fitting %s' % station_id
            
        training, validation, test = split_datasets(df, station_id)      
        t_train = training[pred_col]
        t_validation = validation[pred_col]
        t_test = test[pred_col]
        
        try:            
            # Linear Model
            lm_fit = fit_and_predict_lm(training, validation, test, lr_formula)
            lm_rmse_train = mean_squared_error(t_train, lm_fit[1])**0.5
            lm_rmse_val = mean_squared_error(t_validation, lm_fit[2])**0.5
            lm_rmse_test = mean_squared_error(t_test, lm_fit[3])**0.5
        
            # GAM Model
            gam_fit = fit_and_predict_gam(training, validation, test, gam_formula)
            gam_rmse_train = mean_squared_error(t_train, gam_fit[1])**0.5
            gam_rmse_val = mean_squared_error(t_validation, gam_fit[2])**0.5
            gam_rmse_test = mean_squared_error(t_test, gam_fit[3])**0.5
            
            # Hist Avg
            havg_fit = fit_and_predict_hist_avg(training, validation, test)
            havg_rmse_train = mean_squared_error(t_train, havg_fit[1])**0.5
            havg_rmse_val = mean_squared_error(t_validation, havg_fit[2])**0.5
            havh_rmse_test = mean_squared_error(t_test, havg_fit[3])**0.5
        except Exception as e:
            logging.error(traceback.format_exc())
        
        results.append({'Id': station_id, 
                        'LR-TRAIN-ERR': lm_rmse_train, 'LR-VAL-ERR': lm_rmse_val, 'LR-TEST-ERR': lm_rmse_test,
                        'GAM-TRAIN-ERR': gam_rmse_train, 'GAM-VAL-ERR': gam_rmse_val, 'GAM-TEST-ERR': gam_rmse_test,
                        'HAVG-TRAIN-ERR': havg_rmse_train, 'HAVG-VAL-ERR': havg_rmse_val, 'HAVG-TEST-ERR': havh_rmse_test,#})
                        'LR-MOD': lm_fit[0], 'GAM-MOD': gam_fit[0]})
        
    return results

In [12]:
def convert_results_to_df(results, name):
    dfs = [pd.DataFrame(result).set_index('Id') for result in results]
    for i,df in enumerate(dfs):
        df.columns = ['%s-EXP%d-%s' % (name, i,col) for col in df.columns]
    return pd.concat(dfs, axis=1)

## Use Samples?

In [13]:
stations_to_use = readings.index.get_level_values(0).unique().tolist()

# Short Term Predictions

In [14]:
gam_formula_short = "NbBikes ~ s(TempTMinus2, HumidityTMinus2, bs='tp', sp=30.0) + s(TimeOfDay, by=Weekday, bs='tp', sp=1.1) "  
gam_formula_short += "+ s(TimeOfDay, by=Weekend, bs='tp', sp=50.0) + s(TimeOfDay, by=Holiday, bs='tp', sp=0.2) + s(NbBikesTMinus2, bs='tp', sp=8.0) "
gam_formula_short += "+ s(NbBikesTMinus3, bs='tp', sp=11.0) + RainTMinus2 + FogTMinus2 "

In [15]:
lr_formula_short = "NbBikes ~ TempTMinus2 + HumidityTMinus2 + TimeOfDay + Weekday "
lr_formula_short += "+ NbBikesTMinus2 + NbBikesTMinus3 + RainTMinus2 + FogTMinus2 "

## Baseline

In [16]:
# choose the columns to use in the model
boolean_cols_short = ['Weekday', 'Weekend', 'Holiday', 'RainTMinus2', 'FogTMinus2']
numeric_cols_short = ['HumidityTMinus2', 'TempTMinus2', 'TimeOfDay',
                      'NbBikesTMinus2', 'NbBikesTMinus3', 'HistAvg']                       
pred_col_short = 'NbBikes'

feature_cols_short = numeric_cols_short + boolean_cols_short
cols_short = [pred_col_short] + feature_cols_short

# select the columns chosen columns
readings_short = readings.loc[stations_to_use][cols_short]

# remove na
readings_short.dropna(inplace=True)

In [17]:
baseline_short = [model(readings_short, stations_to_use, gam_formula_short, lr_formula_short, 'NbBikes')]

Fitting BikePoints_1
Fitting BikePoints_10
Fitting BikePoints_100
Fitting BikePoints_101
Fitting BikePoints_102
Fitting BikePoints_103
Fitting BikePoints_104
Fitting BikePoints_105
Fitting BikePoints_106
Fitting BikePoints_107
Fitting BikePoints_108
Fitting BikePoints_109
Fitting BikePoints_11
Fitting BikePoints_110
Fitting BikePoints_111
Fitting BikePoints_112
Fitting BikePoints_113
Fitting BikePoints_114
Fitting BikePoints_115
Fitting BikePoints_116
Fitting BikePoints_117
Fitting BikePoints_118
Fitting BikePoints_119
Fitting BikePoints_12
Fitting BikePoints_120
Fitting BikePoints_121
Fitting BikePoints_122
Fitting BikePoints_123
Fitting BikePoints_124
Fitting BikePoints_125
Fitting BikePoints_126
Fitting BikePoints_127
Fitting BikePoints_128
Fitting BikePoints_129
Fitting BikePoints_13
Fitting BikePoints_130
Fitting BikePoints_131
Fitting BikePoints_132
Fitting BikePoints_133
Fitting BikePoints_134
Fitting BikePoints_135
Fitting BikePoints_136
Fitting BikePoints_137
Fitting BikePoint

In [18]:
baseline_short_df = convert_results_to_df(baseline_short, 'SHORT')

In [19]:
baseline_short_df

,SHORT-EXP0-GAM-TEST-ERR,SHORT-EXP0-GAM-TRAIN-ERR,SHORT-EXP0-GAM-VAL-ERR,SHORT-EXP0-HAVG-TEST-ERR,SHORT-EXP0-HAVG-TRAIN-ERR,SHORT-EXP0-HAVG-VAL-ERR,SHORT-EXP0-LR-TEST-ERR,SHORT-EXP0-LR-TRAIN-ERR,SHORT-EXP0-LR-VAL-ERR
Id,,,,,,,,,
BikePoints_1,0.868446,0.895192,0.829156,4.807993,4.846394,4.315554,0.866595,0.893963,0.828258
BikePoints_10,0.935212,0.962700,0.920177,4.576382,4.456970,4.907174,0.936583,0.964665,0.920177
BikePoints_100,1.019852,1.035073,0.879100,5.029238,5.191576,4.234214,1.019897,1.035073,0.879100
BikePoints_101,1.644451,1.609409,1.798092,4.110624,3.820384,3.998698,1.667633,1.651647,1.801399
BikePoints_102,0.906125,0.895260,0.888641,2.846146,2.880637,2.107715,0.941805,0.938234,0.935679
BikePoints_103,0.769301,0.658364,0.886125,4.201037,4.104669,3.874328,0.770014,0.659477,0.886405
BikePoints_104,1.600881,1.649362,1.747021,6.800566,6.724007,6.153340,1.639654,1.695109,1.792566
BikePoints_105,1.054203,1.028562,1.271638,6.546863,6.423480,6.651657,1.052945,1.033600,1.269296
BikePoints_106,0.815013,0.836485,0.836008,3.415377,3.739926,2.545226,0.837922,0.862313,0.857680


In [20]:
with open("data/parsed/models_short.p", "wb") as f:
    pickle.dump(baseline_short, f)

In [103]:
baseline_short_df.mean()

SHORT-EXP0-GAM-TEST-ERR      1.024929
SHORT-EXP0-GAM-TRAIN-ERR     1.031017
SHORT-EXP0-GAM-VAL-ERR       1.015606
SHORT-EXP0-HAVG-TEST-ERR     5.399333
SHORT-EXP0-HAVG-TRAIN-ERR    5.342800
SHORT-EXP0-HAVG-VAL-ERR      5.228157
SHORT-EXP0-LR-TEST-ERR       1.040086
SHORT-EXP0-LR-TRAIN-ERR      1.050169
SHORT-EXP0-LR-VAL-ERR        1.028567
dtype: float64

## Mid Term Predictions

In [14]:
lr_formula_mid = "NbBikes ~ TempTMinus12 + HumidityTMinus12 + TimeOfDay + Weekday + DistNbBikes + CollNbBikes"
lr_formula_mid += "+ NbBikesTMinus12 + NbBikesTMinus18 + Near1TMinus12 + Near2TMinus12 + Near1TMinus18 + Near2TMinus18 "

In [15]:
gam_formula_mid = "NbBikes ~ s(TempTMinus12, HumidityTMinus12, bs='tp') + s(TimeOfDay, by=Weekday, bs='tp') "
gam_formula_mid += "+ s(TimeOfDay, by=Weekend, bs='tp') + s(TimeOfDay, by=Holiday, bs='tp') + s(NbBikesTMinus12, bs='tp') "
gam_formula_mid += "+ s(NbBikesTMinus18, bs='tp') "

## Baseline

In [16]:
# choose the columns to use in the model
boolean_cols_mid = ['Weekday', 'Weekend', 'Holiday']
numeric_cols_mid = ['HumidityTMinus12', 'TempTMinus12', 'TimeOfDay', 'NbBikesTMinus12', 'NbBikesTMinus18', 'HistAvg', 
                    'Near1TMinus12', 'Near2TMinus12', 'Near1TMinus18', 'Near2TMinus18', 'DistNbBikes', 'CollNbBikes']
pred_col_mid = 'NbBikes'

feature_cols_mid = numeric_cols_mid + boolean_cols_mid
cols_mid = [pred_col_mid] + feature_cols_mid

# select the columns chosen columns
readings_mid = readings.loc[stations_to_use][cols_mid]

# remove na
readings_mid.dropna(inplace=True)

In [17]:
baseline_mid = [model(readings_mid, stations_to_use, gam_formula_mid, lr_formula_mid, 'NbBikes')]

Fitting BikePoints_1
Fitting BikePoints_10
Fitting BikePoints_100
Fitting BikePoints_101
Fitting BikePoints_102
Fitting BikePoints_103
Fitting BikePoints_104
Fitting BikePoints_105
Fitting BikePoints_106
Fitting BikePoints_107
Fitting BikePoints_108
Fitting BikePoints_109
Fitting BikePoints_11
Fitting BikePoints_110
Fitting BikePoints_111
Fitting BikePoints_112
Fitting BikePoints_113
Fitting BikePoints_114
Fitting BikePoints_115
Fitting BikePoints_116
Fitting BikePoints_117
Fitting BikePoints_118
Fitting BikePoints_119
Fitting BikePoints_12
Fitting BikePoints_120
Fitting BikePoints_121
Fitting BikePoints_122
Fitting BikePoints_123
Fitting BikePoints_124
Fitting BikePoints_125
Fitting BikePoints_126
Fitting BikePoints_127
Fitting BikePoints_128
Fitting BikePoints_129
Fitting BikePoints_13
Fitting BikePoints_130
Fitting BikePoints_131
Fitting BikePoints_132
Fitting BikePoints_133
Fitting BikePoints_134
Fitting BikePoints_135
Fitting BikePoints_136
Fitting BikePoints_137
Fitting BikePoint

In [21]:
baseline_mid_df = convert_results_to_df(baseline_mid, 'MID')

In [ ]:
with open("data/parsed/models_mid.p", "wb") as f:
    pickle.dump(baseline_mid, f)

# Long Term Predictions

## All

In [157]:
# choose the columns to use in the model
redistribution_cols = ['CollNbBikesCum6', 'DistNbBikesCum6']
boolean_cols_long = ['Weekday', 'Weekend', 'Holiday']
numeric_cols_long = ['TimeOfDay', 'HistAvg']
pred_col_long = 'NbBikes'

feature_cols_long = numeric_cols_long + boolean_cols_long + redistribution_cols
cols_long = [pred_col_long] + feature_cols_long

# select the columns chosen columns
readings_long = readings.loc[stations_to_use][cols_long]

# remove na
readings_long.dropna(inplace=True)

In [158]:
gam_formula_long = "NbBikes ~ s(TimeOfDay, by=Weekday, bs='tp', sp=35.0) + s(TimeOfDay, by=Weekend, bs='tp', sp=0.7) "
gam_formula_long += "+ s(TimeOfDay, by=Holiday, bs='tp', sp=3.0) + s(HistAvg, bs='tp', k=5) + CollNbBikesCum6 + DistNbBikesCum6 "

In [159]:
lr_formula_long = "NbBikes ~ TimeOfDay + Weekday + Holiday + Weekend + HistAvg + CollNbBikesCum6 + DistNbBikesCum6"

In [160]:
all_long = [model(readings_long, stations_to_use, gam_formula_long, lr_formula_long, 'NbBikes')]

Fitting BikePoints_1
Fitting BikePoints_10
Fitting BikePoints_100
Fitting BikePoints_101
Fitting BikePoints_102
Fitting BikePoints_103
Fitting BikePoints_104
Fitting BikePoints_105
Fitting BikePoints_106
Fitting BikePoints_107
Fitting BikePoints_108
Fitting BikePoints_109
Fitting BikePoints_11
Fitting BikePoints_110
Fitting BikePoints_111
Fitting BikePoints_112
Fitting BikePoints_113
Fitting BikePoints_114
Fitting BikePoints_115
Fitting BikePoints_116
Fitting BikePoints_117
Fitting BikePoints_118
Fitting BikePoints_119
Fitting BikePoints_12
Fitting BikePoints_120
Fitting BikePoints_121
Fitting BikePoints_122
Fitting BikePoints_123
Fitting BikePoints_124
Fitting BikePoints_125
Fitting BikePoints_126
Fitting BikePoints_127
Fitting BikePoints_128
Fitting BikePoints_129
Fitting BikePoints_13
Fitting BikePoints_130
Fitting BikePoints_131
Fitting BikePoints_132
Fitting BikePoints_133
Fitting BikePoints_134
Fitting BikePoints_135
Fitting BikePoints_136
Fitting BikePoints_137
Fitting BikePoint

In [162]:
all_long_df = convert_results_to_df(all_long, 'LONG')

In [86]:
with open("data/parsed/models_long.p", "wb") as f:
    pickle.dump(all_long, f)

NameError: name 'all_long' is not defined

# Present

In [21]:
with open("data/parsed/models_short.p", "rb") as f:
    models_short = pickle.load(f)
with open("data/parsed/models_mid.p", "rb") as f:
    models_mid = pickle.load(f)

In [163]:
short_df = convert_results_to_df(models_short, 'SHORT')
mid_df = convert_results_to_df(models_mid, 'MID')
long_df = convert_results_to_df(all_long, 'LONG')

In [164]:
long_df.describe()

,LONG-EXP0-GAM-TEST-ERR,LONG-EXP0-GAM-TRAIN-ERR,LONG-EXP0-GAM-VAL-ERR,LONG-EXP0-HAVG-TEST-ERR,LONG-EXP0-HAVG-TRAIN-ERR,LONG-EXP0-HAVG-VAL-ERR,LONG-EXP0-LR-TEST-ERR,LONG-EXP0-LR-TRAIN-ERR,LONG-EXP0-LR-VAL-ERR
count,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000,765.000000
mean,5.545743,5.216309,5.736651,5.407190,5.348678,5.239801,5.519201,5.229735,5.651938
std,1.926344,1.777650,2.467123,1.821642,1.843964,2.042587,1.918340,1.779272,2.449580
min,2.294893,2.198929,1.766785,2.229457,2.237461,1.758201,2.294632,2.219586,1.800849
25%,4.182835,3.954905,4.044029,4.110624,4.046713,3.850569,4.167532,3.958752,3.972437
50%,5.229283,4.901323,5.190230,5.123685,5.042512,4.784700,5.224243,4.919091,5.156047
75%,6.541918,6.089167,6.839392,6.384363,6.273090,6.339870,6.515230,6.107812,6.755583
max,16.236486,14.388968,22.562941,14.011610,14.828793,16.309278,16.173550,14.426999,22.077257


In [165]:
short_df['Scenario'] = 'Short-Term'
short_df.set_index('Scenario', append=True, inplace=True)
short_df = short_df.reorder_levels(['Scenario', 'Id'])[['SHORT-EXP0-HAVG-TEST-ERR', 'SHORT-EXP0-LR-TEST-ERR', 'SHORT-EXP0-GAM-TEST-ERR']]
short_df.columns=['HAVG-TEST-ERR', 'LR-TEST-ERR', 'GAM-TEST-ERR']

mid_df['Scenario'] = 'Mid-Term'
mid_df.set_index('Scenario', append=True, inplace=True)
mid_df = mid_df.reorder_levels(['Scenario', 'Id'])[['MID-EXP0-HAVG-TEST-ERR', 'MID-EXP0-LR-TEST-ERR', 'MID-EXP0-GAM-TEST-ERR']]
mid_df.columns=['HAVG-TEST-ERR', 'LR-TEST-ERR', 'GAM-TEST-ERR']

long_df['Scenario'] = 'Long-Term'
long_df.set_index('Scenario', append=True, inplace=True)
long_df = long_df.reorder_levels(['Scenario', 'Id'])[['LONG-EXP0-HAVG-TEST-ERR', 'LONG-EXP0-LR-TEST-ERR', 'LONG-EXP0-GAM-TEST-ERR']]
long_df.columns=['HAVG-TEST-ERR', 'LR-TEST-ERR', 'GAM-TEST-ERR']

In [166]:
aggregations = {
    'HAVG-TEST-ERR': {
        'mean': 'mean',
        'std': 'std'
    },
    'LR-TEST-ERR': {
        'mean': 'mean',
        'std': 'std'
    },
    'GAM-TEST-ERR': {
        'mean': 'mean',
        'std': 'std'
    }
}

In [167]:
rmse_results = pd.concat([short_df, mid_df, long_df])
rmse_results['Metric'] = 'RMSE'

In [168]:
wrmse_results = pd.concat([short_df, mid_df, long_df]).merge(readings.groupby(level='Id')['NbDocks'].mean().to_frame(), how='inner', right_index=True, left_index=True)
wrmse_results['HAVG-TEST-ERR'] = wrmse_results['HAVG-TEST-ERR'] / wrmse_results.NbDocks
wrmse_results['LR-TEST-ERR'] = wrmse_results['LR-TEST-ERR'] / wrmse_results.NbDocks
wrmse_results['GAM-TEST-ERR'] = wrmse_results['GAM-TEST-ERR'] / wrmse_results.NbDocks
wrmse_results['Metric'] = 'WRMSE'
wrmse_results.drop('NbDocks', axis=1, inplace=True)

In [169]:
results = pd.concat([rmse_results, wrmse_results]).reset_index().groupby(['Metric', 'Scenario']).agg(aggregations)
results['HAVG-TEST-ERR', 'HAVG-TEST-ERR'] = results['HAVG-TEST-ERR', 'mean'].map(str) + " (" + results['HAVG-TEST-ERR', 'std'].map(str) + ")"
results['LR-TEST-ERR', 'LR-TEST-ERR'] = results['LR-TEST-ERR', 'mean'].map(str) + " (" + results['LR-TEST-ERR', 'std'].map(str) + ")"
results['GAM-TEST-ERR', 'GAM-TEST-ERR'] = results['GAM-TEST-ERR', 'mean'].map(str) + " (" + results['GAM-TEST-ERR', 'std'].map(str) + ")"
results.columns = results.columns.droplevel()
results = results[['HAVG-TEST-ERR', 'LR-TEST-ERR', 'GAM-TEST-ERR']]
results.index.name = None

In [170]:
results

HAVG-TEST-ERR  \
Metric Scenario                                       
RMSE   Long-Term      5.40719022993 (1.82164249869)   
       Mid-Term        5.4027172564 (1.82225202537)   
       Short-Term     5.39933259074 (1.81863390546)   
WRMSE  Long-Term   0.207424850855 (0.0379533695221)   
       Mid-Term     0.207443103395 (0.039066845878)   
       Short-Term    0.2072065831 (0.0381709451086)   

                                        LR-TEST-ERR  \
Metric Scenario                                       
RMSE   Long-Term      5.51920066142 (1.91833971277)   
       Mid-Term      2.60386000789 (0.993319219906)   
       Short-Term    1.04008594605 (0.409696989696)   
WRMSE  Long-Term   0.211385316337 (0.0397903305265)   
       Mid-Term    0.104578996929 (0.0394646190669)   
       Short-Term   0.04231023103 (0.0183284081812)   

                                        GAM-TEST-ERR  
Metric Scenario                                       
RMSE   Long-Term       5.54574287899 (1.92634384331)  
       Mid-Term        2.37085914916 (0.85810201434)  
       Short-Term     1.02492873261 (0.396915776099)  
WRMSE  Long-Term    0.212344534885 (0.0396780703194)  
       Mid-Term    0.0953142976369 (0.0348835951717)  
       Short-Term  0.0417400851698 (0.0180038973045)

In [121]:
print results.to_latex()

\begin{tabular}{lllll}
\toprule
     &           &                     HAVG-TEST-ERR &                       LR-TEST-ERR &                       GAM-TEST-ERR \\
Metric & Scenario &                                   &                                   &                                    \\
\midrule
RMSE & Long-Term &     5.40096005358 (1.82362774841) &     5.51920066142 (1.91833971277) &      5.53988620589 (1.93049824071) \\
     & Mid-Term &      5.4027172564 (1.82225202537) &    2.60386000789 (0.993319219906) &      2.37085914916 (0.85810201434) \\
     & Short-Term &     5.39933259074 (1.81863390546) &    1.04008594605 (0.409696989696) &     1.02492873261 (0.396915776099) \\
WRMSE & Long-Term &  0.207155675284 (0.0379307023412) &  0.211385316337 (0.0397903305265) &   0.212087492103 (0.0398891487769) \\
     & Mid-Term &   0.207443103395 (0.039066845878) &  0.104578996929 (0.0394646190669) &  0.0953142976369 (0.0348835951717) \\
     & Short-Term &    0.2072065831 (0.0381709451086) &

# Analysis

In [ ]:
with open("data/parsed/r_eval_short.p", "wb") as f:
    pickle.dump(training, f)  

## Short

In [81]:
baseline_short_df['SHORT-EXP0-GAM-TEST-ERR'].sort_values()[765/2:765/2 + 1]

Id
BikePoints_148    0.956842
Name: SHORT-EXP0-GAM-TEST-ERR, dtype: float64

In [82]:
training, validation, test = split_datasets(readings_short, 'BikePoints_148')
with open("data/parsed/r_eval_short.p", "wb") as f:
    pickle.dump(training, f)  

In [99]:
with open("data/parsed/r_eval_short.p", "rb") as f:
    r_eval_short = pickle.load(f)   

In [100]:
%load_ext rpy2.ipython

robjects.globalenv['data'] = r_eval_short

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [101]:
lr_formula_short

'NbBikes ~ TempTMinus2 + HumidityTMinus2 + TimeOfDay + Weekday + NbBikesTMinus2 + NbBikesTMinus3 + RainTMinus2 + FogTMinus2 '

In [102]:
%%R

lrModel <- lm(NbBikes ~ TempTMinus2 + HumidityTMinus2 + TimeOfDay + Weekday + NbBikesTMinus2 + NbBikesTMinus3 + RainTMinus2 + FogTMinus2 , data=data)

summary(lrModel)


Call:
lm(formula = NbBikes ~ TempTMinus2 + HumidityTMinus2 + TimeOfDay + 
    Weekday + NbBikesTMinus2 + NbBikesTMinus3 + RainTMinus2 + 
    FogTMinus2, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-8.9630 -0.1247  0.0250  0.1430  8.8390 

Coefficients: (1 not defined because of singularities)
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)      7.250e-02  1.170e-01   0.619   0.5356    
TempTMinus2     -6.350e-05  3.925e-03  -0.016   0.9871    
HumidityTMinus2  1.041e-03  8.439e-04   1.234   0.2172    
TimeOfDay        2.390e-06  4.427e-07   5.400 6.86e-08 ***
Weekday         -4.372e-02  2.412e-02  -1.812   0.0700 .  
NbBikesTMinus2   1.019e+00  1.581e-02  64.475  < 2e-16 ***
NbBikesTMinus3  -4.041e-02  1.584e-02  -2.552   0.0107 *  
RainTMinus2      7.854e-05  3.484e-02   0.002   0.9982    
FogTMinus2              NA         NA      NA       NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 

In [85]:
%%R

library(mgcv)

gamModel <- gam(NbBikes ~ s(TempTMinus2, HumidityTMinus2, bs='tp', sp=30.0) + s(TimeOfDay, by=Weekday, bs='tp', sp=1.1) 
                + s(TimeOfDay, by=Weekend, bs='tp', sp=50.0) + s(TimeOfDay, by=Holiday, bs='tp', sp=0.2) 
                + s(NbBikesTMinus2, bs='tp', sp=8.0) + s(NbBikesTMinus3, bs='tp', sp=11.0) 
                + RainTMinus2 + FogTMinus2, data=data)

summary(gamModel)


Family: gaussian 
Link function: identity 

Formula:
NbBikes ~ s(TempTMinus2, HumidityTMinus2, bs = "tp", sp = 30) + 
    s(TimeOfDay, by = Weekday, bs = "tp", sp = 1.1) + s(TimeOfDay, 
    by = Weekend, bs = "tp", sp = 50) + s(TimeOfDay, by = Holiday, 
    bs = "tp", sp = 0.2) + s(NbBikesTMinus2, bs = "tp", sp = 8) + 
    s(NbBikesTMinus3, bs = "tp", sp = 11) + RainTMinus2 + FogTMinus2

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  6.84670    0.07532   90.90   <2e-16 ***
RainTMinus2 -0.00211    0.03535   -0.06    0.952    
FogTMinus2   0.00000    0.00000      NA       NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Approximate significance of smooth terms:
                                 edf Ref.df       F p-value    
s(TempTMinus2,HumidityTMinus2) 6.597  8.952   1.003 0.41786    
s(TimeOfDay):Weekday           6.456  7.619 384.035 < 2e-16 ***
s(TimeOfDay):Weekend           2.562  3.033 723.942 < 2e-16 ***
s(Tim

In [86]:
%%R

postscript("reports/images/short-check.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

gam.check(gamModel)

dev.off()


Method: GCV   Optimizer: magic
Model required no smoothing parameter selectionModel rank =  78 / 80 

Basis dimension (k) checking results. Low p-value (k-index<1) may
indicate that k is too low, especially if edf is close to k'.

                                   k'    edf k-index p-value
s(TempTMinus2,HumidityTMinus2) 29.000  6.597   1.002    0.56
s(TimeOfDay):Weekday           10.000  6.456   0.993    0.24
s(TimeOfDay):Weekend           10.000  2.562   0.993    0.33
s(TimeOfDay):Holiday           10.000  5.384   0.993    0.28
s(NbBikesTMinus2)               9.000  3.726   0.994    0.32
s(NbBikesTMinus3)               9.000  2.966   0.993    0.28
png 
  2 


In [87]:
%%R

postscript("reports/images/short-splines.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

layout(matrix(1:2, ncol = 2))
plot(gamModel, scale = 0)
layout(1)

dev.off()

png 
  2 


In [88]:
%%R

postscript("reports/images/short-correlation.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

layout(matrix(1:2, ncol = 2))
acf(resid(gamModel), lag.max = 36, main = "ACF")
pacf(resid(gamModel), lag.max = 36, main = "pACF")
layout(1)

dev.off()

png 
  2 


## Mid

In [89]:
baseline_mid_df['MID-EXP0-GAM-TEST-ERR'].sort_values()[765/2:765/2 + 1]

Id
BikePoints_231    2.224621
Name: MID-EXP0-GAM-TEST-ERR, dtype: float64

In [90]:
training, validation, test = split_datasets(readings_mid, 'BikePoints_231')
with open("data/parsed/r_eval_mid.p", "wb") as f:
    pickle.dump(training, f)  

In [91]:
with open("data/parsed/r_eval_mid.p", "rb") as f:
    r_eval_mid = pickle.load(f)   

In [92]:
%load_ext rpy2.ipython

robjects.globalenv['data'] = r_eval_mid

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [93]:
gam_formula_mid

"NbBikes ~ s(TempTMinus12, HumidityTMinus12, bs='tp') + s(TimeOfDay, by=Weekday, bs='tp') + s(TimeOfDay, by=Weekend, bs='tp') + s(TimeOfDay, by=Holiday, bs='tp') + s(NbBikesTMinus12, bs='tp') + s(NbBikesTMinus18, bs='tp') "

In [94]:
%%R

library(mgcv)

gamModel <- gam(NbBikes ~ s(TempTMinus12, HumidityTMinus12, bs='tp') + s(TimeOfDay, by=Weekday, bs='tp') 
                + s(TimeOfDay, by=Weekend, bs='tp') + s(TimeOfDay, by=Holiday, bs='tp') 
                + s(NbBikesTMinus12, bs='tp') + s(NbBikesTMinus18, bs='tp'), data=data)

summary(gamModel)


Family: gaussian 
Link function: identity 

Formula:
NbBikes ~ s(TempTMinus12, HumidityTMinus12, bs = "tp") + s(TimeOfDay, 
    by = Weekday, bs = "tp") + s(TimeOfDay, by = Weekend, bs = "tp") + 
    s(TimeOfDay, by = Holiday, bs = "tp") + s(NbBikesTMinus12, 
    bs = "tp") + s(NbBikesTMinus18, bs = "tp")

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  11.6694     0.5091   22.92   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Approximate significance of smooth terms:
                                    edf Ref.df       F  p-value    
s(TempTMinus12,HumidityTMinus12) 27.064 28.738   5.207  < 2e-16 ***
s(TimeOfDay):Weekday              9.101  9.576  71.227  < 2e-16 ***
s(TimeOfDay):Weekend              9.322  9.632  19.966  < 2e-16 ***
s(TimeOfDay):Holiday              3.566  4.203  13.835 1.58e-11 ***
s(NbBikesTMinus12)                8.135  8.781 369.863  < 2e-16 ***
s(NbBikesTMinus18)                7.925  8

In [95]:
%%R

postscript("reports/images/mid-check.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

gam.check(gamModel)

dev.off()


Method: GCV   Optimizer: magic
Smoothing parameter selection converged after 21 iterations.
The RMS GCV score gradiant at convergence was 1.054994e-06 .
The Hessian was positive definite.
The estimated model rank was 77 (maximum possible: 78)
Model rank =  77 / 78 

Basis dimension (k) checking results. Low p-value (k-index<1) may
indicate that k is too low, especially if edf is close to k'.

                                     k'    edf k-index p-value
s(TempTMinus12,HumidityTMinus12) 29.000 27.064   0.819    0.00
s(TimeOfDay):Weekday             10.000  9.101   1.010    0.74
s(TimeOfDay):Weekend             10.000  9.322   1.010    0.72
s(TimeOfDay):Holiday             10.000  3.566   1.010    0.72
s(NbBikesTMinus12)                9.000  8.135   0.986    0.12
s(NbBikesTMinus18)                9.000  7.925   0.993    0.30
png 
  2 


In [96]:
%%R

postscript("reports/images/mid-splines.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

layout(matrix(1:2, ncol = 2))
plot(gamModel, scale = 0)
layout(1)

dev.off()

png 
  2 


In [97]:
%%R

postscript("reports/images/mid-correlation.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

layout(matrix(1:2, ncol = 2))
acf(resid(gamModel), lag.max = 36, main = "ACF")
pacf(resid(gamModel), lag.max = 36, main = "pACF")
layout(1)

dev.off()

png 
  2 


## Long

In [171]:
all_long_df['LONG-EXP0-GAM-TEST-ERR'].sort_values()[765/2:765/2 + 1]

Id
BikePoints_610    5.229283
Name: LONG-EXP0-GAM-TEST-ERR, dtype: float64

In [40]:
gam_formula_long

"NbBikes ~ s(TimeOfDay, by=Weekday, bs='tp', sp=35.0) + s(TimeOfDay, by=Weekend, bs='tp', sp=0.7) + s(TimeOfDay, by=Holiday, bs='tp', sp=3.0) + s(HistAvg, bs='tp', k=5) + CollNbBikesCum6 + DistNbBikesCum6 "

In [172]:
training, validation, test = split_datasets(readings_long, 'BikePoints_610')
with open("data/parsed/r_eval_long.p", "wb") as f:
    pickle.dump(training, f)  

In [173]:
with open("data/parsed/r_eval_long.p", "rb") as f:
    r_eval_long = pickle.load(f)   

In [174]:
%load_ext rpy2.ipython

robjects.globalenv['data'] = r_eval_long

In [175]:
%%R

library(mgcv)

gamModel <- gam(NbBikes ~ s(TimeOfDay, by=Weekday, bs='tp', sp=35.0) + s(TimeOfDay, by=Weekend, bs='tp', sp=0.7) 
                + s(TimeOfDay, by=Holiday, bs='tp', sp=3.0) + s(HistAvg, bs='tp', k=5) 
                + CollNbBikesCum6 + DistNbBikesCum6 , data=data)

summary(gamModel)


Family: gaussian 
Link function: identity 

Formula:
NbBikes ~ s(TimeOfDay, by = Weekday, bs = "tp", sp = 35) + s(TimeOfDay, 
    by = Weekend, bs = "tp", sp = 0.7) + s(TimeOfDay, by = Holiday, 
    bs = "tp", sp = 3) + s(HistAvg, bs = "tp", k = 5) + CollNbBikesCum6 + 
    DistNbBikesCum6

Parametric coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)       7.8811     0.5587  14.105  < 2e-16 ***
CollNbBikesCum6   0.3442     0.2183   1.577  0.11490    
DistNbBikesCum6  -0.3834     0.1373  -2.792  0.00526 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Approximate significance of smooth terms:
                       edf Ref.df       F  p-value    
s(TimeOfDay):Weekday 3.250  3.864  11.927 3.41e-09 ***
s(TimeOfDay):Weekend 6.037  7.170  10.201 6.76e-13 ***
s(TimeOfDay):Holiday 3.281  3.852   0.274    0.889    
s(HistAvg)           1.000  1.000 146.366  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Ra

In [177]:
%%R

postscript("reports/images/long-check.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

gam.check(gamModel)

dev.off()


Method: GCV   Optimizer: magic
Smoothing parameter selection converged after 8 iterations.
The RMS GCV score gradiant at convergence was 1.684521e-06 .
The Hessian was positive definite.
The estimated model rank was 36 (maximum possible: 37)
Model rank =  36 / 37 

Basis dimension (k) checking results. Low p-value (k-index<1) may
indicate that k is too low, especially if edf is close to k'.

                        k'   edf k-index p-value
s(TimeOfDay):Weekday 10.00  3.25    1.05    1.00
s(TimeOfDay):Weekend 10.00  6.04    1.05    1.00
s(TimeOfDay):Holiday 10.00  3.28    1.05    1.00
s(HistAvg)            4.00  1.00    0.97    0.02
png 
  2 


In [178]:
%%R

postscript("reports/images/long-splines.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

layout(matrix(1:2, ncol = 2))
plot(gamModel, scale = 0)
layout(1)

dev.off()

png 
  2 


In [179]:
%%R

postscript("reports/images/long-correlation.eps",horizontal=FALSE, paper="special",height=6,width=10, onefile=TRUE)

layout(matrix(1:2, ncol = 2))
acf(resid(gamModel), lag.max = 36, main = "ACF")
pacf(resid(gamModel), lag.max = 36, main = "pACF")
layout(1)

dev.off()

png 
  2 


## Visualization Demo

In [48]:
%run src/data/visualization.py

In [93]:
now = datetime(year=2016, month=6, day=22, hour=9, minute=0, second=0)
single_reading = readings.xs(now, level='Timestamp')

In [102]:
predictions = []
for idx, row in single_reading.iterrows():
    gam = baseline_mid_df.loc[idx]['MID-EXP0-GAM-MOD']
    prediction = gam.predict(row.to_frame().transpose())[0]
    predictions.append({'Prediction': prediction, 'Id': idx, 
                        'NbDocks': row.NbDocks, 
                        'HistAvg': row.HistAvg,
                        'Current': row.NbBikesTMinus12})
predictions = pd.DataFrame(predictions).set_index('Id')

In [103]:
predictions_df = add_station_info(predictions, stations.set_index('Id'), use_indexes=True)
predictions_df['NormalizedPrediction'] = predictions_df.Prediction.astype(float) / predictions_df.NbDocks.astype(float)

In [168]:
from palettable.colorbrewer.diverging import RdYlBu_10

def create_prediction_marker(station):
    
    font_color_now = 'red' if station.Current < 5 else 'black'
    font_color_mid = 'red' if station.Prediction < 5 else 'black'
    font_color_long = 'red' if station.HistAvg < 5 else 'black'
    
    html="""
    <h1>%s</h1>
    <p><strong>Id</strong>: %s</p>
    <p>
    <strong>Bicycle Availability:</strong>
    <ul>
        <font color="%s"><li><strong>%s:</strong> %d bicycles</li></font>
        <font color="%s"><li><strong>%s: </strong> %d bicycles</li></font>
        <font color="%s"><li><strong>%s: </strong> %d bicycles</li></font>
    </ul>
    </p>
    <p>
        <strong>Availability Trend:</strong><br/><br/>
        <img src="http://localhost:8888/files/reports/images/trend.png" width=210 height=210 hspace=16/>
    </p>
    """ % (station.Name, station.name, 
           font_color_now, now.strftime('%d/%m - %H:%M'), station.Current, 
           font_color_mid, (now + timedelta(hours=1)).strftime('%d/%m - %H:%M'), station.Prediction,
           font_color_long, (now + timedelta(hours=3)).strftime('%d/%m - %H:%M'), station.HistAvg)
    iframe = folium.element.IFrame(html=html, width=350, height=530)
    popup = folium.Popup(iframe, max_width=2650)
        
    fill_color = cmap_to_hex(RdYlBu_10.mpl_colormap, station.NormalizedPrediction)
    label = "%s - %s: %d bicycle(s)" 
    
    return folium.CircleMarker(location=[station['Latitude'], station['Longitude']], radius=100,
                        popup=popup, fill_color=fill_color, color=fill_color, fill_opacity=0.99)

predictions_map = draw_stations_map(predictions_df, create_prediction_marker)

In [169]:
folium.TileLayer('stamentoner').add_to(predictions_map)
folium.Map.save(predictions_map, 'reports/maps/predicted_availability_map.html')
predictions_map